In [2]:
import pandas as pd
import cobra
from cobra.flux_analysis.sampling import OptGPSampler

In [3]:
model_dir='iML1515.xml'
model = cobra.io.read_sbml_model(model_dir)
sampling_num = 100000

model.reactions.EX_nh4_e.lower_bound = 0.
model.reactions.EX_3hpp_e.upper_bound = 0.

model.reactions.EX_glc__D_e.lower_bound = -7.25
model.reactions.EX_csn_e.lower_bound = -6.55
model.reactions.EX_ura_e.lower_bound = 6.44

model.reactions.BIOMASS_Ec_iML1515_core_75p37M.lower_bound = model.optimize().objective_value*0.95    

mcmc = OptGPSampler(model, processes = 8)
%time mcmc_sampling = mcmc.sample(sampling_num)

#Save mcmc file
mcmc_value = "mcmc_flux_csn.txt"
mcmc_sampling.to_csv(mcmc_value, sep="\t", header=True)

#Save mcmc head file
mcmc_head = mcmc_sampling.head()
mcmc_head_value = "mcmc_flux_head_csn.txt"
mcmc_head.to_csv(mcmc_head_value, sep="\t", header=True)

#Calculate mcmc mean value
df = mcmc_sampling
mcmc_list = []

for i in mcmc_sampling:
    mcmc_list.append(i)
mcmc_list
mcmc_sum = {}
mcmc_sum_list = []

for j in mcmc_list:
    sums = df[j].sum()
    mean = sums/sampling_num
    mcmc_sum[j] = mean
    for_list = j, mean
    mcmc_sum_list.append(for_list)

index = ['Title', 'mean']
df = pd.DataFrame(mcmc_sum_list, columns = index)
df_inv = df.transpose()
print (df_inv)

#Save mcmc mean value
mcmc_mean_value = "mcmc_mean_flux_csn.txt"
df.to_csv(mcmc_mean_value, sep="\t", header=True)

CPU times: user 17 ms, sys: 37.2 ms, total: 54.2 ms
Wall time: 565 ms
             0          1           2         3        4           5     \
Title      CYTDK2       XPPT       HXPRT     NDPK5   SHK3Dr       NDPK6   
mean   0.00723141  0.0109782  0.00846935 -0.019072  0.23793  0.00245372   

             6          7          8         9      ...            2702  \
Title       NDPK8     DHORTS      OMPDC    PYNP2r   ...    COLIPAabctex   
mean  -0.00704636 -0.0923816  0.0923816 -0.125441   ...     7.98655e-15   

               2703         2704   2705     2706         2707         2708  \
Title  ENLIPAabctex  DM_mththf_c  PYROX  BMOGDS1         MPTS        MOCOS   
mean    5.57131e-05  0.000549738      0        0  4.65753e-18  4.65753e-18   

          2709        2710    2711  
Title  BMOGDS2      FESD2s  OCTNLL  
mean         0  3.5221e-32       0  

[2 rows x 2712 columns]
